# FM(Factorized Machine)因子分解机

该方法常用的场景是推荐，在snippets-rec中会有FM及其衍生算法的介绍。从ml的角度看，该算法不仅仅是一个回归算法, 也可以用于分类、排序等等。

## 1.1 简介
FM可以视为线性回归引入特征交叉的一种演进:

1)线性回归引入特征交叉
$$
\hat{y}(x)=w_0 + \underbrace{\sum_{i=1}^p w_i x_i}_{\text {线性回归 }}+\underbrace{\sum_{i=1}^p \sum_{j=i+1}^p w_{i j} x_i x_j}_{\text {交叉项 (组合待征) }}
$$

其中样本$x_i$为样本x的第$i$维特征，$p$为维数.
但在当x为稀疏数据时（如商品类别或标签）维度可能有几万甚至上百万。交叉项会有维度爆炸，同时绝大多数维度是不相关的，导致交叉项非常稀疏，很难学习。因此FM通过矩阵分解对矩阵$V$进行了近似.

2)FM: 通过矩阵分解对交叉项进行近似
$$
\begin{aligned}
交叉相部分= \\
& \sum_{i=1}^{p} \sum_{j=i+1}^p\left\langle\mathbf{v}_i, \mathbf{v}_j\right\rangle x_i x_j \\
= & \frac{1}{2} \sum_{i=1}^p \sum_{j=1}^p\left\langle\mathbf{v}_i, \mathbf{v}_j\right\rangle x_i x_j-\frac{1}{2} \sum_{i=1}^p\left\langle\mathbf{v}_i, \mathbf{v}_i\right\rangle x_i x_i \\
= & \frac{1}{2}\left(\sum_{i=1}^p \sum_{j=1}^p \sum_{f=1}^k v_{i, f} v_{j, f} x_i x_j-\sum_{i=1}^p \sum_{f=1}^k v_{i, f} v_{i, f} x_i x_i\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{i=1}^{p} v_{i, f} x_i\right)\left(\sum_{j=1}^{p} v_{j, f} x_j\right)-\sum_{i=1}^{p} v_{i, f}^2 x_i^{2}\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left ( \sum_{i=1}^{p} v_{i, f} x_i \right)^{2}-\sum_{i=1}^{p} v_{i, f}^2 x_i^{2}\right)
\end{aligned}
$$


上述为y关于x的分布的表达式（还缺一个残差项，如正态分布的残差）。
下面我们将上述表达式以样本形式矩阵化，方便机器学习和深度学习引擎求解，通过下面几步阐述：
step 1)假定共有$n$个样本，即$X \in \mathbb{R}^{n \times p}$，$Y \in \mathbb{R}^{n}$
$$
X = \begin{bmatrix}
x_1^{(1)} & \dots & x_p^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
x_1^{(n)} & \dots & x_p^{(n)} \\
\end{bmatrix}
$$

step 2)$V \in \mathbb{R}^{p \times k}$
$$
V = \begin{bmatrix}
v_1^{(1)} & \dots & v_k^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
v_1^{(p)} & \dots & v_k^{(p)} \\
\end{bmatrix}
$$

step 3)Y的交叉项$Y_{inter}$

$$
Y_{inter} = \sum_{i=1}^{p} \sum_{j=i+1}^{p} \langle \textbf v_i, \textbf v_j \rangle \vec{x_i} \odot \vec{x_j}  \\
= \frac{1}{2} \sum_{f=1}^{k} \Big( \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2 - \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2 \Big) \\
= \frac{1}{2} \sum_{f=1}^{k}  \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2 -
\frac{1}{2} \sum_{f=1}^{k}  \big( \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2 \big)
$$

注意：
- 此处$\vec{x_i} $表示向量，$\vec{x_i} \in \mathbb{R}^{n \times 1}$
- $\odot$表示元素乘法(element wise product)，此处平方运算$\vec{x_i}^{2}$也表示向量的元素级平方操作.
- 记住该公式相减的一部分和第二部分, 后续会用到.

step4)Y的交叉项可以利用$XV$进一步简化:
$X$和$V$的内积($n*k$维):
$$
XV = \begin{bmatrix}
\sum_{i=1}^{p} v_f^{(1)} x_i^{(1)}  & \dots &  \sum_{i=1}^{p} v_f^{(k)} x_i^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
\sum_{i=1}^{p} v_f^{(1)} x_i^{(n)} & \dots & \sum_{i=1}^{(n)} v_f^{(k)} x_i^{(n)} \\
\end{bmatrix} =
\begin{bmatrix}
S_{1,1}^{(1)}  & \dots &  S_{1,k}^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
S_{1,1}^{(n)}  & \dots & S_{1,k}^{(n)} \\
\end{bmatrix}
$$



我们注意到:
- step3中$Y$求解第一项中求和元素$\sum_{i=1}^{p} v_f^{(i)}$和 $ \vec{x_i} $ 的乘积恰好等于$XV$的第$i$列:
$$
v_f^{(i)} \vec{x_i} = \begin{pmatrix}
  v_f^{(i)} x_i^{(1)}  \\
  ... \\
  v_f^{(i)} x_i^{(n)}
\end{pmatrix}
$$

- 因此step3中第一项$\frac{1}{2} \sum_{f=1}^{k}  \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2$等价于$XV$的元素级平方$(XV)^2 = (XV) \odot (XV)$.
我们定义一个运算表示对矩阵的列求和: $\sigma_{(1)}(B)$表示对矩阵$B$的列向量求和. 因此有:
$$
\frac{1}{2} \sum_{f=1}^{k}  \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2
=\frac{1}{2} \sigma_{(1)}((XV) \odot (XV))
$$

- 同理step3中的第二项中求和元素$ \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2$可以视为$(X \odot X) \dot (V \odot V)$的第$i$列, 第二项可以视为$(X \odot X) \dot (V \odot V)$的列向量求和:

$$
\frac{1}{2} \sum_{f=1}^{k}  \big( \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2 \big)
=\frac{1}{2} \sigma_{(1)}((X \odot X) \dot (V \odot V))
$$

因此最终的矩阵表达式为:

$$
Y = w_0 + X w + \frac{1}{2} \sigma_{(1)}((XV) \odot (XV)) - \frac{1}{2} \sigma_{(1)}((X \odot X) \dot (V \odot V))
$$
其中:
- $w_0$为截距项;
- $w$为线性项系数;
- $V$为交叉项的隐式特征;
- $\odot$为元素级乘法;
- $\sigma_{(1)}(B)$表示对矩阵$B$的列向量求和


In [ ]:
import torch


class TorchFM(torch.nn.Module):
    def __init__(self, n=None, k=None):
        super().__init__()
        self.V = torch.nn.Parameter(torch.randn(n, k), requires_grad=True)
        self.linear = torch.nn.Linear(n, 1)

    def forward(self, x):
        out_1 = torch.matmul(x, self.V).pow(2).sum(1, keepdim=True)  #S_1^2
        out_2 = torch.matmul(x.pow(2), self.V.pow(2)).sum(1, keepdim=True)  # S_2

        out_inter = 0.5 * (out_1 - out_2)
        out_linear = self.linear(x)
        out = out_inter + out_linear

        return out

In [2]:
import torch
vv = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [3]:
vv.pow(2)

tensor([[ 1,  4,  9],
        [16, 25, 36],
        [49, 64, 81]])